# Sync rules on GitHub with VT

Author: @JohnLaTwC

This notebook uses the VT API to synchronize a set of Yara rules on github (or another website) with your VT hunting rules.

API: https://github.com/VirusTotal/vt-py/

Docs: https://virustotal.github.io/vt-py/

In [1]:
!!pip install vt-py
!!pip install nest_asyncio

['Requirement already satisfied: nest_asyncio in c:\\anaconda3\\lib\\site-packages (1.4.0)']

In [2]:
import vt 
import nest_asyncio
nest_asyncio.apply()

def get_rules(url):
    import requests
    
    rules = None
    r = requests.get(url)
    if r.status_code==404:
        print('rules do not exist')
    else:
        rules = r.content.decode()
    return rules

def get_ruleset_id(api_key, rule_name):
    with vt.Client(api_key) as client:
        obj = client.get_json('/intelligence/hunting_rulesets',
                             params = {'filter':'enabled:true name:%s ' % rule_name, 'limit':1})
        if len(obj['data']) == 0:
            return -1
        return obj['data'][0]['id']

      
def create_update_rule(api_key, rule_name, vt_covid_rule):
    with vt.Client(api_key) as client:
        
        id = get_ruleset_id(api_key, rule_name)

        rs = vt.Object("hunting_ruleset")
        rs.name = rule_name
        rs.enabled = True
        rs.rules = vt_covid_rule
        res = None
        if id == -1:
            res = client.post_object("/intelligence/hunting_rulesets", obj=rs)
        else:
            res = client.patch_object("/intelligence/hunting_rulesets/%s" % id, obj=rs)
        if 'id' in res.to_dict():
            print("Success.")
        else:
            print("Failed.")

In [ ]:
github_url = 'https://raw.githubusercontent.com/JohnLaTwC/Shared/master/Yara/TemplateRules.yara'
#github_url = 'https://raw.githubusercontent.com/Neo23x0/signature-base/master/yara/apt_lazarus_aug20.yar'

#VT_APIKEY = "..."

from urllib.parse import urlparse
rule_name = 'imported_rules' + urlparse(github_url).path.split('/')[-1].split('.yar')[0]

create_update_rule(VT_APIKEY, rule_name, get_rules(github_url))